In [1]:
import numpy as np
import copy
from scipy.ndimage import imread
from scipy import misc
import matplotlib.pyplot as plt
import os
import re
import tensorflow as tf

C:\Users\james\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
originPath = os.getcwd()
pattern = re.compile(".png$")
dirList=list(filter(lambda x: os.path.isdir(x) and not re.search(pattern, x) , os.listdir()))

In [3]:
def imageGet(currentpath, dictFile, encode,endswith):
    dirList= list(filter(lambda x: os.path.isdir(x) , os.listdir()))
    if len(dirList) < 1:
        listFile = list(filter(lambda x: x.endswith(endswith) , os.listdir()))
        if len(listFile) >0:
            path = os.getcwd().split("\\")[-2:]
            path='-'.join(path)
            FileList = list(map(lambda x: os.path.join(os.getcwd(), x) ,listFile))
            labels=[len(encode) for i in range(len(FileList))]
            encode[path] = len(encode)
            dictFile.append(tuple(zip(labels, FileList)))
        return dictFile
    for i in dirList:
        changePath = os.path.join(currentpath, i)
        os.chdir(changePath)
        imageGet(changePath, dictFile,encode , endswith)

In [4]:
dirList

['.ipynb_checkpoints',
 'images_background',
 'images_background_small1',
 'images_background_small2']

In [5]:
FilePaths = []
encoder = {}
imageGet(os.getcwd(), FilePaths, encoder, ".png")
classNum = len(encoder)
os.chdir(originPath)

In [6]:
FilePathsNp = np.array(FilePaths)
FilePathsNp=np.reshape(FilePathsNp,(FilePathsNp.shape[0]*FilePathsNp.shape[1], FilePathsNp.shape[2]))
np.random.shuffle(FilePathsNp)
FilePathsNp.shape

(25120, 2)

In [7]:
classNum

964

In [8]:
encoderedLists = FilePathsNp[:,0].astype(int)
imagePaths = FilePathsNp[:,1]

In [9]:
encoderedLists
depth = len(encoderedLists)
encodered =tf.one_hot(encoderedLists, classNum)
print(encodered.shape)

(25120, 964)


In [10]:
photos=[misc.imread(x,flatten=True) for x in imagePaths]

C:\Users\james\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.


In [11]:
dfphoto = np.array(photos)
dfphoto = np.expand_dims(dfphoto,3)

In [12]:
k=3
channels=1
dim_hidden=64
stride = [1,2,2,1]
dtype = tf.float32
conv_initializer =  tf.contrib.layers.xavier_initializer_conv2d(dtype=dtype)
fc_initializer =  tf.contrib.layers.xavier_initializer(dtype=dtype)
data_tf = tf.convert_to_tensor(dfphoto, np.float32)
data_tf =tf.expand_dims(data_tf,3)
print(data_tf.shape)
# data_tf = tf.reshape(data_tf, (-1, data_tf.shape[0],data_tf.shape[1], data_tf.shape[2]))
# print(data_tf.shape)

(25120, 105, 105, 1, 1)


In [13]:
weights = {}
weights['conv1'] = tf.get_variable('conv1', [k, k, channels, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b1'] = tf.Variable(tf.zeros([dim_hidden]))
weights['conv2'] = tf.get_variable('conv2', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b2'] = tf.Variable(tf.zeros([dim_hidden]))
# weights['conv3'] = tf.get_variable('conv3', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
# weights['b3'] = tf.Variable(tf.zeros([dim_hidden]))
# weights['conv4'] = tf.get_variable('conv4', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
# weights['b4'] = tf.Variable(tf.zeros([dim_hidden]))

In [14]:
def covnNet(input,weight, bais):
    conv_output = tf.nn.conv2d(input, weight, stride, 'SAME') + bais
    return tf.nn.relu(conv_output)

In [15]:
ImageContextHolder = tf.placeholder(tf.float32, [None, data_tf.shape[1], data_tf.shape[2],data_tf.shape[3]])
OneHotContextHolder = tf.placeholder(tf.float32, [None, classNum])

hidden = covnNet(ImageContextHolder, weights['conv1'],weights['b1'] )
hidden = covnNet(hidden, weights['conv2'],weights['b2'] )
# hidden = covnNet(hidden, weights['conv3'],weights['b3'] )
# hidden = covnNet(hidden, weights['conv4'],weights['b4'] )

hidden = tf.contrib.layers.flatten(hidden)

weights['w5'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1], tf.int32) , classNum]),dtype=dtype, name='w5')
weights['b5'] = tf.Variable(tf.zeros([classNum]), name='b5')

weights['w6_1'] = tf.Variable(tf.random_normal([classNum,128]),dtype=dtype, name='w6_1')
weights['w6_2'] = tf.Variable(tf.random_normal([classNum,128]),dtype=dtype, name="w6_2")
weights['b6'] = tf.Variable(tf.zeros([128*2]), name='b6')

hidden = tf.matmul(weights['w5'],weights['w6_1'])
hidden = tf.matmul(OneHotContextHolder,weights['w6_2'])
hidden = tf.concat([weights['w6_1'],weights['w6_2']],axis=1)
print(hidden.shape)
hidden = tf.nn.relu(tf.add(hidden, weights['b6'] ))

weights['w7'] = tf.Variable(tf.random_normal([128*2,128]),dtype=dtype, name="w6_2")
weights['b7'] = tf.Variable(tf.zeros([128]), name='b7')
hidden = tf.add(tf.matmul(hidden, weights['w7']),weights['b7'])

representation = tf.reduce_mean(hidden, axis=1)
print(representation.shape)

(964, 256)
(964,)


In [16]:
representation=tf.expand_dims(representation,1)

In [17]:
ImageTargetHolder = tf.placeholder(tf.float32, [None, data_tf.shape[1], data_tf.shape[2],data_tf.shape[3]])
OneHotTargetHolder = tf.placeholder(tf.float32, [None,classNum])

weights['conv1_T'] = tf.get_variable('conv1_T', [k, k, channels, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b1_T'] = tf.Variable(tf.zeros([dim_hidden]))
weights['conv2_T'] = tf.get_variable('conv2_T', [k, k, dim_hidden, dim_hidden], initializer=conv_initializer, dtype=dtype)
weights['b2_T'] = tf.Variable(tf.zeros([dim_hidden]))
hidden = covnNet(ImageTargetHolder, weights['conv1_T'],weights['b1_T'] )
hidden = covnNet(hidden, weights['conv2_T'],weights['b2_T'] )


In [18]:
hidden = tf.contrib.layers.flatten(hidden)
weights['w3_T'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1], tf.int32) , tf.cast(representation.shape[0],tf.int32)]),dtype=dtype, name='w3_T')
weights['b3_T'] = tf.Variable(tf.zeros([tf.cast(representation.shape[0],tf.int32)]), name='b3_T')
weights['w4_T'] = tf.Variable(tf.random_normal([tf.cast(representation.shape[0],tf.int32) ,128]),dtype=dtype, name='w4_T')
hidden = tf.add(tf.matmul(hidden,weights['w3_T']), weights['b3_T'])
hidden = tf.concat([weights['w4_T'],representation],axis=1)
weights['b4_T'] = tf.Variable(tf.zeros([tf.cast(hidden.shape[1],tf.int32)]), name='b4_T')
hidden = tf.add(hidden, weights['b4_T'])

weights['w5_T'] = tf.Variable(tf.random_normal([tf.cast(hidden.shape[1],tf.int32) , classNum]),dtype=dtype, name='w5_T')
weights['b5_T'] = tf.Variable(tf.zeros([classNum]), name='b5_T')

hidden = tf.add(tf.matmul(hidden,weights['w5_T']), weights['b5_T'])
hidden = tf.reduce_mean(hidden, axis=1)

In [19]:
# hidden=tf.nn.sigmoid(hidden)

In [20]:
hidden = tf.expand_dims(hidden,0)

In [21]:
hidden

<tf.Tensor 'ExpandDims_2:0' shape=(1, 964) dtype=float32>

In [22]:
OneHotTargetHolder

<tf.Tensor 'Placeholder_3:0' shape=(?, 964) dtype=float32>

In [23]:
loss=tf.nn.sigmoid_cross_entropy_with_logits(logits=hidden, labels=OneHotTargetHolder) 

In [24]:
predict=tf.argmax(hidden, axis=1)
print(predict)

Tensor("ArgMax:0", shape=(1,), dtype=int64)


In [25]:
loss= tf.reduce_mean(loss)

In [26]:
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

In [27]:
encodered.dtype

tf.float32

In [28]:
TRAINING_ITERATIONS= 10000
# imagePaths
# encoderedLists
contextNum = int(depth * 0.5)
contextPic = dfphoto[:contextNum,:,:]
contextlabel = encodered[:contextNum]

targetPic= dfphoto[contextNum:,:,:]
targetlabel = encodered[contextNum:]

In [29]:
targetlabel.shape

TensorShape([Dimension(12560), Dimension(964)])

In [30]:
OneHotContextHolder.shape

TensorShape([Dimension(None), Dimension(964)])

In [31]:
with tf.Session() as sess:
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic, OneHotTargetHolder: targetlabel.eval()}
    sess.run(tf.global_variables_initializer())
    for it in range(TRAINING_ITERATIONS):
        _,loss_reduce =sess.run([train_step, loss], feed_dict)
        if (it % 1000 == 0):
            print(loss_reduce)
    print(loss_reduce)
    print(loss_reduce.shape)
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic, OneHotTargetHolder: targetlabel.eval()}
    predictions=sess.run([predict], feed_dict)
    print(predictions)

0.7071268
0.042439085
0.010441296
0.0073499084
0.0067550945
0.0065921103
0.006507603
0.0064562005
0.006434632
0.006428434
0.0064270385
()
[array([752], dtype=int64)]


In [32]:
contextPic = dfphoto[:contextNum,:,:]
contextlabel = encodered[:contextNum]
targetPic= dfphoto[0,:,:]
targetlabel = encodered[0]
targetPic=tf.expand_dims(targetPic,0)
targetlabel=tf.expand_dims(targetlabel,0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    feed_dict = {ImageContextHolder: contextPic, OneHotContextHolder: contextlabel.eval(),  ImageTargetHolder: targetPic.eval(), OneHotTargetHolder: targetlabel.eval()}
    predictions=sess.run([hidden], feed_dict)
    print(predictions)
    print(encodered[0].eval())
    print(encoderedLists[0])

[array([[-2.95333236e-01,  8.23479220e-02, -2.44572997e-01,
         6.73138648e-02,  8.57659876e-02, -1.61860302e-01,
         1.23427831e-01, -7.97426477e-02, -1.61051244e-01,
         1.54945999e-01,  3.59517246e-01, -6.72322333e-01,
        -7.80989900e-02, -2.09968507e-01,  2.23389063e-02,
        -3.07804734e-01,  1.03308745e-01, -1.26040787e-01,
         3.82409841e-01, -7.62322471e-02, -6.57886744e-01,
        -4.33038734e-02, -4.01795171e-02, -2.81498060e-02,
        -3.51898104e-01,  2.78089374e-01,  3.02017659e-01,
        -3.11641902e-01, -2.31983885e-02, -3.75099093e-01,
        -8.78283903e-02, -3.17843556e-01, -1.59895778e-01,
         1.03498353e-02,  5.55650055e-01,  1.32533826e-03,
         1.32844076e-01, -1.38928778e-02, -8.32167193e-02,
        -2.83110365e-02,  2.26198062e-01, -4.67404127e-01,
         1.26928955e-01,  3.01153570e-01,  1.34887069e-01,
         3.77546787e-01,  3.50055933e-01,  3.28230232e-01,
        -4.99608405e-02,  1.04800321e-01, -2.81903595e-

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [34]:
predictions[0][0][257]

-0.094719976